<h1>Algorithmic Trading Course</h1>

<h3>SECTION 01: Algorithmic Trading Basics</h3>

Algorithmic trading means using computers to mae invstment decisions.

There are many different types of algorithmic trading. The main difference is their spee of execution.

Here are some of the main players in the algorthmic trading landscape:
- Renaissance Technologies: $165Bin AUM
- AQR Capital Management: $61B in AUM
- Citadel Securities: $32B in AUM

<h4> The algorithmic Trading Process:</h4>
The process of running a quantitative investing strategy can be broken down into the following steps:<br>

1. Collect Data
2. Develope a hypohesis for a strategy
3. Backtest that strategy
4. Implement the strategy in production

<h3>SECTION 02: API Basics and Course Configuration</h3>

<h4> What is an API? </h4>
An API is an Application Programing Interface. It allow us to interact with someone else's software.<br>
We are going to use IEX Cloud API to access database of financial data.<br><br/>
In this course will be using the GET request to gather data from the IEX Cloud API.

Other API Functionality:
- POST: Adds data to the database exposed by the API. (Create only)
- PUT: Adds and verviewrites data in the database exposed by the API. (Create or replace)
- DELETE: Deletes data from the API's database.

----------

### PROJECT 01 - Equal-Weight S&P 500 Screener

The S&P 500 is the world's most popular stock market index.<br>
Many investment funds are benchmarked to the S&P 500. This means that they seek to replicate the performance of this index by owning all stocks that are held in the index.<br>
The S&P covers the 500 largest companies of United States. 

In the first project you will build an alternative version of the S&P 500 that assigns the same wheight to each company.<br>
The goal of this project is to create a Python script that accepts the value on your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

<h4>Importing libraries</h4>

In [1]:
import numpy as np
import pandas as pd 
import requests 
import xlsxwriter
import math

<h4> Importing Our List of Stocks</h4>
The stock in S&P 500 change over time, they are re-balanced.So in the real world you would connect to the Standard and Poors provider and pull their real-time database.<br></br>

For now we are going to use this static version of S&P. [Click this link to download them now](http://nickmccullum.com/algorithmic-trading-python/sp_500_stocks.csv). 

### PROJECT 02 - Quantitative Momentum Screener

<h3> Momentum Investing Project Overview</h3>
Momentum investing means investing in assets that have increased in price the most.</br>
Consider this example to better understand:<br>
Imagine that you have the choice between investing in two stocks that have had the following returns over the last year: 

- Apple(APPL): 35%
- Microsoft (MSFT): 20%

A momentum investing strategy would suggest investing in Apple because of its higher recent price return. <br>
There are many other nuances to momentum investing strategies.

### PROJECT 03 - Quantitative Value Investing Strategy Screener

<h3> Value Investing Project Overview</h3>
Value investing means investing in stocks that are trading below their perceived intrinsic value.<br>
Value investing was popularized by investors like Waren Buffet, seth Klarman, and  Benjamin Graham.<br></br>
Creating algorithmic value investing srtategies relies on a concept called <font color=red>multiples</font>.<br> 
Multiples are simply a way that investors use to estimate how valuable a company is.<br>
Multiples are calculated by dividing a company's stock price by sme measure of company's worth like earnings or assets.

Examples of <font color=red>multiples</font> used in value investing:

- Price-to-earnings
- Price-to-book-value
- Price-to-free-cash-flow

Each of the individual multilpes used by value investors has its pros and cons. <br>
One way to minimize the impact of any specific multiples is by using a <font color=red>composite</font>.<br></br>
Composite is just a average of many different valuation strategies. <br>
We'll use  composite of 5 different value investing metrics.